<a href="https://colab.research.google.com/github/biranchi2018/face_recognition_single_image/blob/master/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install face_recognition

    100% |████████████████████████████████| 100.2MB 305kB/s 
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [0]:
import face_recognition
import cv2
import os
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
!ls

dataset_faces.dat  sample_data


In [6]:
cv2.__version__

'3.4.3'